# This part 3 of 3 is completely TBD very soon

**Previous notebooks:** <br>
[Part 1: Data mining and munging](tdp_1_3_data_mining_and_cleaning.ipynb)<br>
[Part 2: Data stats and visuals](tdp_2_3_data_statistics_visualization.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns; sns.set()

### Data loading

In [2]:
# Main data for modeling
biker_class = pd.read_csv('preprocessed/biker_class.csv', index_col=0)

# Additional parts for statistics
biker_stats = pd.read_csv('preprocessed/biker_stats.csv', index_col=0)
biker_by_stage = pd.read_csv('preprocessed/biker_by_stage.csv', index_col=0)
tour_results = pd.read_csv('preprocessed/tours.csv', index_col=False)
stage_results = pd.read_csv('preprocessed/stages.csv', index_col=False)
stage_class = pd.read_csv('preprocessed/stages_summary.csv', index_col=False)

# Data modeling

In [ ]:
# TBD